# Library

In [52]:
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from scipy import stats
import xgboost as xgb
import matplotlib.pyplot as plt


# Read data from csv file

In [51]:
df = pd.read_csv('data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by=['row', 'col'], ascending=[True, True]).reset_index(drop=True)
features = [
    'AWS2', 'CAPE', 'V850', 'EWSS', 'KX', 'U250', 'U850', 'CIN', 'V250', 'R250',
    'hour_sin','hour_cos','doy_sin','doy_cos'
]
target = 'AWS'
df['AWS2'] = df['AWS']

for i in features[0:10]:
    df = clean(i,df)

df['year'] = df['datetime'].dt.year
df['hour']     = df['datetime'].dt.hour
df['doy']      = df['datetime'].dt.dayofyear
df['hour_sin'] = np.sin(2*np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2*np.pi * df['hour'] / 24)
df['doy_sin']  = np.sin(2*np.pi * df['doy']  / 365)
df['doy_cos']  = np.cos(2*np.pi * df['doy']  / 365)


dftrain = df[df['datetime'] <= pd.Timestamp('2020-10-15 23:00:00')].reset_index(drop=True)
dftest = df[df['datetime'] >= pd.Timestamp('2020-10-16 00:00:00')].reset_index(drop=True)
print(len(df))
scaler_f = StandardScaler()
dftrain[features] = scaler_f.fit_transform(dftrain[features])
dftest[features] = scaler_f.fit_transform(dftest[features])
dftest.head()

657681


,row,col,datetime,AWS,CAPE,CIN,EWSS,IE,ISOR,KX,...,V250,V850,AWS2,year,hour,doy,hour_sin,hour_cos,doy_sin,doy_cos
0,0,104,2020-10-01 00:00:00,0.0,197.250,61.9535,-113.0550,-0.000017,0.676296,36.6214,...,-0.157303,3.68979,0.0,2020,0,275,0.000000,1.000000,-0.999769,0.021516
1,0,104,2020-10-01 01:00:00,0.0,138.500,9999.0000,-128.2850,-0.000038,0.676296,36.0785,...,-0.572311,3.51732,0.0,2020,1,275,0.258819,0.965926,-0.999769,0.021516
2,0,104,2020-10-01 02:00:00,0.0,102.000,9999.0000,-120.6720,-0.000055,0.676296,35.4105,...,-1.284910,3.18250,0.0,2020,2,275,0.500000,0.866025,-0.999769,0.021516
3,0,104,2020-10-01 03:00:00,0.0,160.750,9999.0000,-169.1910,-0.000063,0.676296,34.8431,...,-2.056780,2.46533,0.0,2020,3,275,0.707107,0.707107,-0.999769,0.021516
4,0,104,2020-10-01 04:00:00,0.0,204.500,213.4690,-304.3480,-0.000112,0.676296,34.8934,...,-2.280150,1.55983,0.0,2020,4,275,0.866025,0.500000,-0.999769,0.021516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657676,87,136,2020-10-31 19:00:00,7.0,49.000,9999.0000,-80.7422,-0.000012,0.451531,33.1501,...,0.735031,-1.85147,7.0,2020,19,305,-0.965926,0.258819,-0.858764,0.512371
657677,87,136,2020-10-31 20:00:00,6.0,36.625,9999.0000,-55.3477,-0.000011,0.451531,33.4893,...,0.769287,-2.38234,6.0,2020,20,305,-0.866025,0.500000,-0.858764,0.512371
657678,87,136,2020-10-31 21:00:00,3.8,43.625,9999.0000,-54.0625,-0.000009,0.451531,33.4570,...,0.791824,-2.82980,3.8,2020,21,305,-0.707107,0.707107,-0.858764,0.512371
657679,87,136,2020-10-31 22:00:00,2.4,34.875,9999.0000,-50.9336,-0.000012,0.451531,33.2179,...,0.880249,-3.69713,2.4,2020,22,305,-0.500000,0.866025,-0.858764,0.512371


In [ ]:
scaler_f = StandardScaler()
dftrain[features] = scaler_f.fit_transform(dftrain[features])
dftest[features] = scaler_f.transform(dftest[features])

In [57]:
def make_sequences(data, feats, tgt, window_size, horizon):
    X, y = [], []
    arr_f = data[feats].values
    arr_t = data[tgt].values
    time = data['datetime'].tolist()
    for i in range(window_size, len(data)-horizon+1):
        check = False
        for j in range(i-window_size+1, i+horizon):
            if (time[j] - time[j-1]).total_seconds() != 3600:
                check = True
                break
        if check:
            continue
        X.append(arr_f[i-window_size:i])
        y.append(arr_t[i:i+horizon])
    return np.array(X), np.array(y)


In [57]:
window_size = 1
horizon     = 6

X_train, y_train = make_sequences(dftrain, features, target, window_size, horizon)
X_test, y_test = make_sequences(dftest, features, target, window_size, horizon)
print("Train shapes:", X_train.shape, y_train.shape)
print("Test shapes:", X_test.shape, y_test.shape)


301935
355746
Train shapes: (227476, 1, 14) (227476, 6)


# XGboost

In [53]:
xgbr_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

In [55]:
xgbr_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [60]:
y_pred = xgbr_model.predict(X_test.reshape(-1,10))
load_model = xgbr_model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
mae = mean_absolute_error(y_test,y_pred)
print("Mean Absolute Error:", mae)
mape = mean_absolute_percentage_error(y_test,y_pred)
print("Mean Absolute Percentage Error:", mae)
r2 = r2_score(y_test,y_pred)
print("R-squared:", r2)

Mean Squared Error: 323.1598416849608
Mean Absolute Error: 14.83225671081034
Mean Absolute Percentage Error: 14.83225671081034
R-squared: -49.88894193972931


# Lstm

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
lstm_scores, xgb_scores = [], []

for tr, vl in tscv.split(X_train):
    X_tr, X_vl = X_train[tr], X_train[vl]
    y_tr, y_vl = y_train[tr], y_train[vl]
    
    # --- LSTM với EarlyStopping ---
    model_l = Sequential([
        LSTM(50, input_shape=(window_size,len(features))),
        Dense(horizon)
    ])
    model_l.compile('adam','mse')
    es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
    model_l.fit(X_tr, y_tr, epochs=50, callbacks=[es], verbose=0)
    p_l = model_l.predict(X_vl)
    lstm_scores.append(np.sqrt(mean_squared_error(y_vl, p_l)))
    
    # --- XGBoost ---
    X_tr_f = X_tr.reshape(len(X_tr), -1)
    X_vl_f = X_vl.reshape(len(X_vl), -1)
    model_x = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1
    )
    model_x.fit(X_tr_f, y_tr, verbose=False)
    p_x = model_x.predict(X_vl_f)
    xgb_scores.append(np.sqrt(mean_squared_error(y_vl, p_x)))

l_rmse = np.mean(lstm_scores)
x_rmse = np.mean(xgb_scores)
w_l = 1/l_rmse
w_x = 1/x_rmse

print(f"LSTM CV RMSE:   {l_rmse:.4f}")
print(f"XGB   CV RMSE:   {x_rmse:.4f}")
print(f"Ensemble weights → LSTM: {w_l:.2f}, XGB: {w_x:.2f}")


In [ ]:
# — LSTM final —
lstm_final = Sequential([
    LSTM(50, input_shape=(window_size,len(features))),
    Dense(horizon)
])
lstm_final.compile('adam','mse')
es = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
lstm_final.fit(X_train, y_train, epochs=50, callbacks=[es], verbose=1)

# — XGBoost final —
Xf = X_train.reshape(len(X_train), -1)
xgb_final = xgb.XGBRegressor(
    objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1
)
xgb_final.fit(Xf, y_train)


In [ ]:
import pickle

class Ensemble:
    def __init__(self, lstm_model, xgb_model, w_l, w_x):
        self.lstm = lstm_model
        self.xgb  = xgb_model
        self.w_l  = w_l
        self.w_x  = w_x

    def predict(self, X):
        y1 = self.lstm.predict(X)
        flat = X.reshape(len(X), -1)
        y2 = self.xgb.predict(flat).reshape(y1.shape)
        return (self.w_l * y1 + self.w_x * y2) / (self.w_l + self.w_x)

ensemble = Ensemble(lstm_final, xgb_final, w_l, w_x)

with open('ensemble_final_2.pkl', 'wb') as f:
    pickle.dump(ensemble, f)

print("Saved ensemble model to 'ensemble_final.pkl'")


In [ ]:
import pickle
ens = pickle.load(open('ensemble_final_2.pkl','rb'))
X_test, y_test = make_sequences(df2020, features, target, window_size, horizon)
y_pred = ens.predict(X_test)
print('MSE', mean_squared_error(y_test, y_pred))

In [ ]:
test_day   = pd.to_datetime('2020-04-15 00:00')
prev_times = [test_day - timedelta(hours=i) for i in range(window_size,0,-1)]


grid = df2020[ coord_cols ].drop_duplicates().reset_index(drop=True)
nP   = len(grid)

pred_map = np.full((horizon, nP), np.nan)
act_map  = np.full((horizon, nP), np.nan)

for idx, pt in grid.iterrows():
    # lấy subset cho điểm đó
    cond = True
    for c in coord_cols:
        cond &= (df2020[c] == pt[c])
    df_loc = df2020[cond].sort_values('datetime').reset_index(drop=True)
    
    # window 3h input
    df_win = df_loc[df_loc['datetime'].isin(prev_times)]
    if len(df_win) != window_size:
        continue
    
    Xd = df_win[features].values.reshape(1, window_size, len(features))
    y_l = lstm_final.predict(Xd).flatten()
    y_x = xgb_final.predict(Xd.reshape(1,-1)).flatten()
    # ensemble trọng số
    y_e = (w_l*y_l + w_x*y_x) / (w_l + w_x)
    pred_map[:, idx] = y_e
    
    # actual 6h
    df_act = df_loc[
        (df_loc['datetime'] >= test_day) &
        (df_loc['datetime'] <  test_day + timedelta(hours=horizon))
    ].sort_values('datetime')
    if len(df_act) != horizon:
        continue
    act_map[:, idx] = df_act[target].values

print(f"Built maps for {nP} points.")


In [ ]:
import matplotlib.pyplot as plt
vmin = np.nanmin(act_map)
vmax = np.nanmax(act_map)

fig, axes = plt.subplots(nrows=horizon, ncols=2, figsize=(10, 4*horizon))
for h in range(horizon):
    ax1, ax2 = axes[h]
    ax1.scatter(grid[coord_cols[1]], grid[coord_cols[0]],
                c=act_map[h], cmap='viridis', vmin=vmin, vmax=vmax, s=20)
    ax1.set_title(f'Actual map in {h+1}h')
    ax2.scatter(grid[coord_cols[1]], grid[coord_cols[0]],
                c=pred_map[h], cmap='viridis', vmin=vmin, vmax=vmax, s=20)
    ax2.set_title(f'Predicted map in {h+1}h')
    xmn, xmx = grid[coord_cols[1]].min(), grid[coord_cols[1]].max()
    ymn, ymx = grid[coord_cols[0]].min(), grid[coord_cols[0]].max()
    ax1.set_xlim(xmn-0.5, xmx+0.5);  ax1.set_ylim(ymn-0.5, ymx+0.5)
    ax2.set_xlim(xmn-0.5, xmx+0.5);  ax2.set_ylim(ymn-0.5, ymx+0.5)

fig.colorbar(axes[0,0].collections[0], ax=axes, orientation='vertical', fraction=0.02)
plt.tight_layout()
plt.show()


In [ ]:
# Cell 11: Metrics & line chart Mean Actual vs Mean Predicted
mask = ~np.isnan(act_map) & ~np.isnan(pred_map)
all_act = act_map[mask]
all_prd = pred_map[mask]

rmse = np.sqrt(mean_squared_error(all_act, all_prd))
mae  = mean_absolute_error(all_act, all_prd)
r2   = r2_score(all_act, all_prd)
print(f"Test Day {test_day.date()} → RMSE={rmse:.3f}, MAE={mae:.3f}, R²={r2:.3f}")

mean_act = np.nanmean(act_map, axis=1)
mean_prd = np.nanmean(pred_map, axis=1)
hours = np.arange(1, horizon+1)

plt.figure(figsize=(8,4))
plt.plot(hours, mean_act, marker='o', label='Mean Actual')
plt.plot(hours, mean_prd, marker='x', label='Mean Predicted')
plt.xlabel('Hour ahead')
plt.ylabel('AWS')
plt.title('Mean Actual vs Mean Predicted across grid')
plt.legend()
plt.show()
